In [1]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email

In [2]:
jubilee = JubileeMotionController()
jubilee.gcode('M98 P"/sys/config.g"')


'ok\n'

In [3]:
jubilee.home_all()

In [ ]:
jubilee.move_xyz_relative(z=100)

In [5]:
jubilee.axes_homed

[True, True, True, True]

In [36]:
jubilee.gcode("M18")

''

In [5]:
camera = camera_tool(jubilee)

In [14]:
camera.uninstall()

In [7]:
posicao1 = [165.0, 211.0, 300.0]
posicao2 = [165.0, 316.0, 300.0]
posicoes = (posicao1,posicao2)

In [10]:
n = 1000


for i in range(n):
    placa = ('placa 1','placa 2')
    for posicao,p in zip(posicoes,placa):
        now = datetime.now()

        current_time_hms = now.strftime("%H:%M:%S")
        x,y,z = posicao
        jubilee.move_xyz_absolute(x=x,y=y,z=z,velocity=8000)
        time.sleep(2)
        last_save = f'fotos_teste/{i} {p} {current_time_hms}.jpg'
        camera.photo(last_save,video_index=0)
        print(i,end='\r')

    if i%10 == 0:
        send_email("Teste_30_07",f"Iteração:{i}",image_path=last_save)
    
    time.sleep(1)

KeyboardInterrupt: 